In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.5 MB/s eta 0:00:00


In [3]:
# --- Mohamed's Side ---
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import hashlib

In [4]:
# Generate Mohamed's RSA keys
mohamed_key = RSA.generate(2048)
mohamed_private = mohamed_key
mohamed_public = mohamed_key.publickey()

In [5]:
# Save Mohamed's public key
with open("mohamed_pub.pem", "wb") as f:
    f.write(mohamed_public.export_key())

In [6]:
# Load Alaa's public key (exchange manually)
alaa_public = RSA.import_key(open("alaa_pub.pem", "rb").read())

In [7]:
# Generate Mohamed's random AES part
mohamed_random = get_random_bytes(16)

In [8]:
# Encrypt Mohamed's AES part with Alaa's public key
cipher_rsa_alaa = PKCS1_OAEP.new(alaa_public)
encrypted_mohamed_random = cipher_rsa_alaa.encrypt(mohamed_random)

In [9]:
# Save to send to Alaa
with open("encrypted_mohamed_key.bin", "wb") as f:
    f.write(encrypted_mohamed_random)

In [10]:
# Load Alaa's encrypted part (after exchange)
encrypted_alaa_random = open("encrypted_alaa_key.bin", "rb").read()

In [11]:
# Decrypt Alaa's part with Mohamed's private key
alaa_random = PKCS1_OAEP.new(mohamed_private).decrypt(encrypted_alaa_random)

In [12]:
# Generate session key
session_key = hashlib.sha256(mohamed_random + alaa_random).digest()[:16]

In [13]:
# Encrypt message and sign
message = b"MohamedID202201647"
cipher = AES.new(session_key, AES.MODE_EAX)
nonce = cipher.nonce
ciphertext, tag = cipher.encrypt_and_digest(message)

with open("encrypted_message.bin", "wb") as f:
    f.write(nonce + tag + ciphertext)

signature = pkcs1_15.new(mohamed_private).sign(SHA256.new(message))
with open("signature.bin", "wb") as f:
    f.write(signature)